## Import libraries and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./warehouse_short.csv')

## Explore data and do a little data cleaning

In [3]:
df.head()

,id,vehicle_id,unix_timestamp,latitude,longitude,odometer,vehicle_speed,clutch_switch,brake_switch,engine_speed,...,ignition_time,is_coupled_to_another_vehicle,road_type,surface_type,elevation,openweather_api_id,temperature,humidity,wind_speed,wind_degree
0,1,x1sdt0ier,1663261470,53.50895,9.898597,50163.0,0.0,NaN,99.0,500.0,...,NaN,NaN,service,NaN,5,NaN,NaN,NaN,NaN,NaN
1,2,x1sdt0ier,1663261531,53.50893,9.898541,50163.0,0.0,NaN,99.0,500.0,...,NaN,NaN,service,NaN,5,NaN,NaN,NaN,NaN,NaN
2,3,x1sdt0ier,1663261593,53.50891,9.898545,50163.0,0.0,NaN,99.0,499.0,...,NaN,NaN,service,NaN,5,NaN,NaN,NaN,NaN,NaN
3,4,x1sdt0ier,1663261654,53.50891,9.898636,50163.0,0.0,NaN,99.0,501.0,...,NaN,NaN,service,NaN,5,NaN,NaN,NaN,NaN,NaN
4,5,x1sdt0ier,1663261714,53.50892,9.898651,50163.0,0.0,NaN,99.0,499.0,...,NaN,NaN,service,NaN,5,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             100 non-null    int64  
 1   vehicle_id                     100 non-null    object 
 2   unix_timestamp                 100 non-null    int64  
 3   latitude                       100 non-null    float64
 4   longitude                      100 non-null    float64
 5   odometer                       14 non-null     float64
 6   vehicle_speed                  100 non-null    float64
 7   clutch_switch                  0 non-null      float64
 8   brake_switch                   14 non-null     float64
 9   engine_speed                   14 non-null     float64
 10  is_moving                      100 non-null    object 
 11  axle_load                      0 non-null      float64
 12  fuel_level                     100 non-null    floa

Taking into account that there are so many columns without information, I'll delete it for the data analysis part.

In [5]:
df.columns

Index(['id', 'vehicle_id', 'unix_timestamp', 'latitude', 'longitude',
       'odometer', 'vehicle_speed', 'clutch_switch', 'brake_switch',
       'engine_speed', 'is_moving', 'axle_load', 'fuel_level',
       'engine_coolant_temperature', 'total_fuel_used', 'total_engine_hours',
       'avg_fuel_consumption', 'gps_accuracy', 'ignition_time',
       'is_coupled_to_another_vehicle', 'road_type', 'surface_type',
       'elevation', 'openweather_api_id', 'temperature', 'humidity',
       'wind_speed', 'wind_degree'],
      dtype='object')

In [6]:
df.drop(columns=['clutch_switch', 'axle_load', 'total_engine_hours',
                 'avg_fuel_consumption', 'gps_accuracy', 'ignition_time',
                 'is_coupled_to_another_vehicle', 'openweather_api_id',
                 'temperature', 'humidity','wind_speed', 'wind_degree'], inplace = True)

In [7]:
df.head()

,id,vehicle_id,unix_timestamp,latitude,longitude,odometer,vehicle_speed,brake_switch,engine_speed,is_moving,fuel_level,engine_coolant_temperature,total_fuel_used,road_type,surface_type,elevation
0,1,x1sdt0ier,1663261470,53.50895,9.898597,50163.0,0.0,99.0,500.0,f,89.0,87.0,14797.0,service,NaN,5
1,2,x1sdt0ier,1663261531,53.50893,9.898541,50163.0,0.0,99.0,500.0,f,89.0,87.0,14797.0,service,NaN,5
2,3,x1sdt0ier,1663261593,53.50891,9.898545,50163.0,0.0,99.0,499.0,f,89.0,87.0,14798.0,service,NaN,5
3,4,x1sdt0ier,1663261654,53.50891,9.898636,50163.0,0.0,99.0,501.0,f,89.0,87.0,14798.0,service,NaN,5
4,5,x1sdt0ier,1663261714,53.50892,9.898651,50163.0,0.0,99.0,499.0,f,89.0,87.0,14798.0,service,NaN,5


Check for uniqueness of values and delete those variables where there are not enough variability of data (if I can, I'll try to make a estimation of null values). Taking into account the information below, I'll delete the columns 'brake_switch' (it's difficult to make a estimation with so less data and zero variability) and 'is_moving' (also, it doesn't make sense the information from this last field: if the vehicle speed is not 0, then it should be moving):

In [8]:
len(df['vehicle_id'].unique())

41

In [9]:
df.describe()

,id,unix_timestamp,latitude,longitude,odometer,vehicle_speed,brake_switch,engine_speed,fuel_level,engine_coolant_temperature,total_fuel_used,elevation
count,100.000000,1.000000e+02,100.000000,100.000000,14.000000,1.000000e+02,14.0,14.000000,100.000000,14.00000,14.000000,100.000000
mean,50.500000,1.663262e+09,53.438467,9.752635,44615.071429,3.186464e+01,99.0,606.642857,63.124000,85.50000,13091.857143,34.960000
std,29.011492,8.247939e+01,1.510136,2.452888,15737.198683,3.770966e+01,0.0,163.691834,23.651165,2.82162,5120.239911,54.902654
min,1.000000,1.663261e+09,48.237061,4.019217,25257.000000,0.000000e+00,99.0,499.000000,16.400000,82.00000,6826.000000,0.000000
25%,25.750000,1.663262e+09,52.348856,8.951527,25257.000000,9.854212e-16,99.0,500.000000,46.300000,82.00000,6826.000000,5.000000
50%,50.500000,1.663262e+09,53.508925,9.968837,50163.000000,2.808065e+00,99.0,571.500000,64.000000,87.00000,14797.500000,23.500000
75%,75.250000,1.663262e+09,53.581818,11.266658,58948.500000,8.324211e+01,99.0,600.000000,79.400000,87.00000,17793.500000,53.250000
max,100.000000,1.663262e+09,56.554554,13.641021,61878.000000,8.408414e+01,99.0,1018.000000,102.000000,90.00000,18792.000000,495.000000


In [10]:
df['is_moving'].value_counts()

f    100
Name: is_moving, dtype: int64

In [11]:
df['road_type'].value_counts()

service           42
motorway          22
unclassified       6
path               5
primary            4
track              4
cycleway           4
secondary          3
motorway_link      2
rest_area          2
footway            2
tertiary           1
proposed           1
secondary_link     1
residential        1
Name: road_type, dtype: int64

In [12]:
df['surface_type'].value_counts()

asphalt          46
dirt              3
paved             2
paving_stones     2
concrete          2
unpaved           1
fine_gravel       1
Name: surface_type, dtype: int64

In [13]:
df.drop(columns=['brake_switch', 'is_moving'], inplace = True)

### Estimations

- Odometer and total fuel used values: it can be difficult to estimate them taking into account that the first one indicates the total distance that the vehicle has ran. It could be estimated maybe based on the total fuel used, but the only data we have are from only three vehicles (and it's the same data that we only have available for the odometer), so maybe the best is to delete both columns

In [14]:
df[(pd.isnull(df['odometer']) == False)]

,id,vehicle_id,unix_timestamp,latitude,longitude,odometer,vehicle_speed,engine_speed,fuel_level,engine_coolant_temperature,total_fuel_used,road_type,surface_type,elevation
0,1,x1sdt0ier,1663261470,53.50895,9.898597,50163.0,0.0,500.0,89.0,87.0,14797.0,service,NaN,5
1,2,x1sdt0ier,1663261531,53.50893,9.898541,50163.0,0.0,500.0,89.0,87.0,14797.0,service,NaN,5
2,3,x1sdt0ier,1663261593,53.50891,9.898545,50163.0,0.0,499.0,89.0,87.0,14798.0,service,NaN,5
3,4,x1sdt0ier,1663261654,53.50891,9.898636,50163.0,0.0,501.0,89.0,87.0,14798.0,service,NaN,5
4,5,x1sdt0ier,1663261714,53.50892,9.898651,50163.0,0.0,499.0,89.0,87.0,14798.0,service,NaN,5
5,6,3o1e4e0qf,1663261520,53.51216,9.901346,61877.0,49.0,1018.0,44.0,90.0,18792.0,secondary,asphalt,5
6,7,3o1e4e0qf,1663261581,53.51280,9.899107,61878.0,16.0,933.0,44.0,88.0,18792.0,service,NaN,6
7,8,3o1e4e0qf,1663261642,53.51279,9.898967,61878.0,0.0,500.0,44.0,87.0,18792.0,service,NaN,6
8,9,3o1e4e0qf,1663261703,53.51280,9.899117,61878.0,0.0,544.0,44.0,87.0,18792.0,service,NaN,6
9,10,3tq3s6p4g,1663261495,53.51277,9.899006,25257.0,0.0,599.0,70.0,82.0,6826.0,service,NaN,6


In [15]:
df[(pd.isnull(df['total_fuel_used']) == False)]

,id,vehicle_id,unix_timestamp,latitude,longitude,odometer,vehicle_speed,engine_speed,fuel_level,engine_coolant_temperature,total_fuel_used,road_type,surface_type,elevation
0,1,x1sdt0ier,1663261470,53.50895,9.898597,50163.0,0.0,500.0,89.0,87.0,14797.0,service,NaN,5
1,2,x1sdt0ier,1663261531,53.50893,9.898541,50163.0,0.0,500.0,89.0,87.0,14797.0,service,NaN,5
2,3,x1sdt0ier,1663261593,53.50891,9.898545,50163.0,0.0,499.0,89.0,87.0,14798.0,service,NaN,5
3,4,x1sdt0ier,1663261654,53.50891,9.898636,50163.0,0.0,501.0,89.0,87.0,14798.0,service,NaN,5
4,5,x1sdt0ier,1663261714,53.50892,9.898651,50163.0,0.0,499.0,89.0,87.0,14798.0,service,NaN,5
5,6,3o1e4e0qf,1663261520,53.51216,9.901346,61877.0,49.0,1018.0,44.0,90.0,18792.0,secondary,asphalt,5
6,7,3o1e4e0qf,1663261581,53.51280,9.899107,61878.0,16.0,933.0,44.0,88.0,18792.0,service,NaN,6
7,8,3o1e4e0qf,1663261642,53.51279,9.898967,61878.0,0.0,500.0,44.0,87.0,18792.0,service,NaN,6
8,9,3o1e4e0qf,1663261703,53.51280,9.899117,61878.0,0.0,544.0,44.0,87.0,18792.0,service,NaN,6
9,10,3tq3s6p4g,1663261495,53.51277,9.899006,25257.0,0.0,599.0,70.0,82.0,6826.0,service,NaN,6


In [16]:
df.drop(columns = ['odometer', 'total_fuel_used'], inplace = True)

In [17]:
pd.set_option('display.max_rows', 100)

- Surface type: maybe the best indicator of surface could be given by road type, so we will replace null values by the mode per group of road and surface type. Afterwards, for the rest of null values, we will just replace them with 'Unknown'

In [18]:
a = df[['road_type', 'surface_type', 'id']].groupby(['road_type', 'surface_type']).count().reset_index()
b = a.groupby(['road_type'])['id'].max().reset_index().set_index(['road_type', 'id'])
a.set_index(['road_type', 'id'], inplace = True)

In [19]:
a = a[a.index.isin(b.index)].reset_index().drop(columns = ['id'])

In [20]:
df[['road_type','surface_type']] = df[['road_type','surface_type']].set_index(['road_type'])['surface_type'].fillna(a.set_index(['road_type'])['surface_type']).reset_index()

In [21]:
df['surface_type'].fillna('unknown', inplace = True)

- Engine speed and engine coolant temerature: we will drop them because a estimation of its values can be a bit risky:
    - The correlation is only based on 14 out of 100 cases (14%), which can be considered low
    - Correlation is medium in the best of the cases (-.63 between engine speed and fuel level), it isn't as high as the 80-90% to be confident enough about the prediction
    - I also tried to estimate the engine coolant temperature with the latitude, but it isn't very correlated

In [22]:
np.corrcoef(df[pd.isnull(df['engine_coolant_temperature']) == False]['elevation'], df[pd.isnull(df['engine_coolant_temperature']) == False]['engine_coolant_temperature'])

array([[ 1.        , -0.63702206],
       [-0.63702206,  1.        ]])

In [23]:
np.corrcoef(df[pd.isnull(df['engine_coolant_temperature']) == False]['elevation'], df[pd.isnull(df['engine_coolant_temperature']) == False]['engine_coolant_temperature'])

array([[ 1.        , -0.63702206],
       [-0.63702206,  1.        ]])

In [24]:
np.corrcoef(df[pd.isnull(df['engine_coolant_temperature']) == False]['elevation'], df[pd.isnull(df['engine_coolant_temperature']) == False]['engine_coolant_temperature'])

array([[ 1.        , -0.63702206],
       [-0.63702206,  1.        ]])

In [25]:
df.drop(columns = ['engine_speed', 'engine_coolant_temperature'], inplace = True)

## Checking for duplicates across different fields

In [26]:
df.shape

(100, 10)

In [27]:
df.drop_duplicates().shape

(100, 10)

## Questions regarding data engineering part

### T1 - What tools would you use for data engineering here in general?

I would use DBT (the tool I know the most for ETL processes) to extract the information I need to calculate every KPI.

### T2 - Which tools would you apply for a, b and c individually considering each would have individual requirements in terms of how the data is used? You are free to assume a certain constraint from your own experience and solve this question.

I don't understand very well this question, so I'll answer it based on 

For task a, I'll obtain the data referred to consumption per km as calculated in the data analysis part of this notebook (i.e. only using the columns referred to 'vehicle_id', 'vehicle_speed', 'unix_timestamp', 'fuel_consumption'), as well as the data that inform us about driving conditions such as type of road (if the type of road is similar, then the consumption could be similar as well), latitude and longitude (if drivers are within the same geographical area, the driving conditions should be more similar compared to those which are in different zones), as well as other data not available in this dataset (e.g. climate conditions, type of vehicle used, demographic characteristics of drivers...).

To calculate the number of trips made, I'd use the data about latitude, longitude and also speed: we can calculate the number of trips that have been made if at some point, the speed is 0, increases and then decreases again to 0, latitude and longitude have changed enough. Also, to differentiate between trips, at least there should be 10 minutes where the vehicle has been stopped (i.e. with vehicle speed being 0): that way, we will differentiate between two different trips vs the usual stops a driver has to make to be respectful with driving signals and other indications. The fourth point of this set of tasks can be calculated with the information obtained along this process, as well as the usage profile.

For task b, if the data about Co2 emissions isn't available, we can create an ETL process that calculates it based on the consumption of fuel, the type of fuel consumed and the vehicle brand and model. That way, we can create a table within the databases that has, epro trip, the desired characteristics (trip id, timstamp of the beginning of the trip, timestamp of its ending, driver id, vehicle id among other characteristics). In order to compare the performance before and after the app, we can ask for historical data of trips to the company that hires the services of the app; if it isn't available, then we can make a estimation based on similar data we could have within our databases from other clients.

Finally, for task c, the data needed would be a mix between process a and b, where it'd be necessary to have data related to fuel consumption, latitude and longitude trips, timestamp for each latitude and longitude points. After calculating the KPI 'fuel consumption', we could use a notebook or any other visualization tool to represent the decrease over time per concrete vehicle or aggregating the measure by other characteristics (branch, model and branch...). The same can be done for the second point of this task: after calculating the KPI 'distance per fleet', we could represent it as well. The ñast points can be calculated based on historical or estimated data before and after using CO2OPT services.

### T3 - Write a sample code using Airflow to enable:

##### a. Data injestion from a sample endpoint: http://localhost:3000/random-endpoint-1 with a JSON payload into a dataware connected to Snowflake

Useful resources:

    - https://towardsdatascience.com/apache-airflow-for-data-science-how-to-work-with-rest-apis-8f4e20bee7d
    
    - https://airflow.apache.org/docs/apache-airflow-providers-http/stable/_api/airflow/providers/http/sensors/http/index.html
    
    - https://airflow.apache.org/docs/apache-airflow/1.10.7/_api/airflow/hooks/http_hook/index.html
    
    - https://airflow.apache.org/docs/apache-airflow/stable/howto/connection.html
    
I have commented all the code for this part in order to run propperly all the data analysis part for the notebook.

In [28]:
# import json
# from datetime import datetime
# from airflow.models import DAG
# from airflow.providers.http.sensors.http import HttpSensor
# from airflow.providers.http.operators.http import SimpleHttpOperator
# from airflow.operators.python import PythonOperator
# from airflow.models.connection import Connection
# from snowflake.connector import connect

# ## API connection to retrieve data
# connection_api = Connection(
#     conn_id = 'default_connection',
#     host = 'http://localhost',
#     port = 3000
#     login = 'x',
#     password = 'y',
#     extra = json.dumps(dict(this_param= 'some val', that_param = 'other val*')),
# )

# ## Snowflake connection to store data
# connection_snowflake = connect(
#     host=host,
#     user=user,
#     password=password,
#     account=account,
#     warehouse=warehouse,
#     database=database,
#     schema=schema,
#     protocol='https',
#     port=port
# )

# snowflake_query = [
#     '''insert into database {}'''.format(ti.xcom_pull(task_ids=['get_posts'])),
#     ]

# ## Airflow pipeline
# with DAG(
#     dag_id = 'extract_endpoint_into_dataware',
#     schedule_interval = '@daily',
#     start_date = datetime(2022, 12, 3),
#     catchup = False
# ) as dag:
    
#     # 1. Check if the API is up
#     task_is_api_active = HttpSensor(
#         task_id = 'is_api_active',
#         http_conn_id = connection_api,
#         endpoint = 'random-endpoint-1/'
#     )
    
#     # 2. Get the posts
#     task_get_data = SimpleHttpOperator(
#         task_id = 'get_posts',
#         http_conn_id = connection_api,
#         endpoint = 'random-endpoint-1/',
#         method = 'GET',
#         response_filter = lambda response: json.loads(response.text),
#         log_response = True
#     )
    
#     # 3. Save the posts into database in Snowflake
#     create_insert = SnowflakeOperator(
#         task_id = "insert_into_db",
#         sql = snowflake_query,
#         snowflake_conn_id = connection_snowflake,
#     )
    
#     task_is_api_active >> task_get_data >> create_insert

##### b. Orchestrating data analytics to enable KPI calculations for either a/b/c (only 1)

All these KPIs could be calculated as in the section "Questions regarding the data analysis part > product team".

### T4 - Explain in 5-7 sentences, what kind of pre-processing strategies and tools have you used in order to ensure the trueness of your analytics?

Assuming that there are not outliers, the process to ensure that data had enough quality has been:
1. Explore data nature (variable types, number of missing values)
2. Check if there were variables with all missing values
3. Determine if there were variables with some missing values and if they could be guessed:

    3.1. If they could be guessed, then fill them
    
    3.2. If thay couldn't be guessed and there were a lot of missing values (more than the 50% of the data), then delete the variable

4. Check for duplicate values

## Questions regarding the data analysis part

### Product team

#### 1. Fuel consumption per km

Calculate fuel consumption per kilometer in those vehicles that have data about their fuel level, and also where that fuel level decreases or is kept the same over time.

In [29]:
fuel_consumption = df.groupby(['vehicle_id']).agg({'fuel_level': [np.min, np.max]}).reset_index()

In [30]:
fuel_consumption.columns = ['vehicle_id', 'min', 'max']

In [31]:
fuel_consumption[fuel_consumption['min'] != fuel_consumption['max']].head()

,vehicle_id,min,max
6,3tq3s6p4g,70.0,72.0
8,478v3j0qq,16.4,17.2
17,aof3f34ok,71.6,72.0
20,f5uo7nhgj,48.4,48.8
27,lta7bpqtk,25.6,26.0


In [32]:
subset_fuel_consumption = df[df['vehicle_id'].isin(fuel_consumption[fuel_consumption['min'] != fuel_consumption['max']]['vehicle_id'].unique())]

In [33]:
subset_fuel_consumption['next_unix_timestamp'] = subset_fuel_consumption.groupby(['vehicle_id'])['unix_timestamp'].shift(-1)
subset_fuel_consumption['previous_fuel_level'] = subset_fuel_consumption.groupby(['vehicle_id'])['fuel_level'].shift(1)

C:\Users\garji\AppData\Local\Temp\ipykernel_5924\3583146522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_fuel_consumption['next_unix_timestamp'] = subset_fuel_consumption.groupby(['vehicle_id'])['unix_timestamp'].shift(-1)
C:\Users\garji\AppData\Local\Temp\ipykernel_5924\3583146522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_fuel_consumption['previous_fuel_level'] = subset_fuel_consumption.groupby(['vehicle_id'])['fuel_level'].shift(1)


In [34]:
subset_fuel_consumption = subset_fuel_consumption[(subset_fuel_consumption['next_unix_timestamp'] > subset_fuel_consumption['unix_timestamp']) &
                                                  (subset_fuel_consumption['fuel_level'] <= subset_fuel_consumption['previous_fuel_level'])
                                                 ]

In [35]:
subset_fuel_consumption['delta_seconds'] = subset_fuel_consumption['next_unix_timestamp'] - subset_fuel_consumption['unix_timestamp']
subset_fuel_consumption['delta_fuel'] =  subset_fuel_consumption['previous_fuel_level'] - subset_fuel_consumption['fuel_level']

Assumptions: Vehicle speed provided in km / h and fuel level in liters. Then:

$$
km = speed (km/h) * time(hours)
$$

In [36]:
subset_fuel_consumption['km'] = subset_fuel_consumption['vehicle_speed'] * subset_fuel_consumption['delta_seconds'] / 3600

In [37]:
subset_fuel_consumption = subset_fuel_consumption[subset_fuel_consumption['km'] > 0]

In [38]:
subset_fuel_consumption_global = subset_fuel_consumption.groupby(['vehicle_id']).sum()[['delta_fuel', 'km']].reset_index()

C:\Users\garji\AppData\Local\Temp\ipykernel_5924\2835586945.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subset_fuel_consumption_global = subset_fuel_consumption.groupby(['vehicle_id']).sum()[['delta_fuel', 'km']].reset_index()


In [39]:
print('Avg. fuel consumption (litres) per km: {}'.format(np.mean(subset_fuel_consumption_global['delta_fuel']/subset_fuel_consumption_global['km'])))

Avg. fuel consumption (litres) per km: 0.26292901406010166


#### 2. Comparison to other vehicles observed with similar driving conditions

In [40]:
avg_consumption = np.mean(subset_fuel_consumption_global['delta_fuel']/subset_fuel_consumption_global['km'])

##### 2.1. Average consumption per type of road

In [41]:
subset_fuel_consumption['consumption'] = subset_fuel_consumption['delta_fuel'] / subset_fuel_consumption['km']

In [42]:
subset_fuel_consumption_road_type = subset_fuel_consumption.groupby(['road_type']).sum()[['delta_fuel', 'km']].reset_index()

C:\Users\garji\AppData\Local\Temp\ipykernel_5924\1514744343.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subset_fuel_consumption_road_type = subset_fuel_consumption.groupby(['road_type']).sum()[['delta_fuel', 'km']].reset_index()


In [43]:
subset_fuel_consumption_road_type['consumption'] = subset_fuel_consumption_road_type['delta_fuel'] / subset_fuel_consumption_road_type['km']

In [44]:
a = subset_fuel_consumption.copy()
a = a.merge(subset_fuel_consumption_road_type[['road_type', 'consumption']], how = 'inner', left_on = ['road_type'], right_on = ['road_type'])

In [45]:
a['comparison_per_road_type'] = a[['road_type','consumption_x', 'consumption_y']].apply(lambda x: 'Greater than average of surface {}'.format(x['road_type'])
                                                                                           if x['consumption_x'] > x['consumption_y'] 
                                                                                                   else (
                                                                                                    'Same as average of surface {}'.format(x['road_type'])
                                                                                                       if x['consumption_x'] == x['consumption_y'] 
                                                                                                       else 'Lower than average of surface {}'.format(x['road_type']))
                                                                                           , axis = 1)

In [46]:
a

,id,vehicle_id,unix_timestamp,latitude,longitude,vehicle_speed,fuel_level,road_type,surface_type,elevation,next_unix_timestamp,previous_fuel_level,delta_seconds,delta_fuel,km,consumption_x,consumption_y,comparison_per_road_type
0,16,f5uo7nhgj,1663261586,55.042694,11.987868,83.168006,48.4,tertiary,asphalt,44,1.663262e+09,48.8,117.0,0.4,2.702960,0.147986,0.147986,Same as average of surface tertiary
1,27,pqf1ffjoi,1663261585,51.933884,4.074568,81.000854,53.6,service,paved,8,1.663262e+09,53.6,123.0,0.0,2.767529,0.000000,0.000000,Same as average of surface service
2,42,478v3j0qq,1663261585,52.683624,13.343811,81.124029,16.4,motorway,asphalt,72,1.663262e+09,17.2,123.0,0.8,2.771738,0.288628,0.141611,Greater than average of surface motorway
3,45,lta7bpqtk,1663261585,53.549400,10.688777,83.541390,25.6,motorway,asphalt,19,1.663262e+09,25.6,124.0,0.0,2.877537,0.000000,0.141611,Lower than average of surface motorway
4,56,rg1f1f4wq,1663261588,51.448326,6.740520,41.345348,53.6,primary,asphalt,27,1.663262e+09,54.8,119.0,1.2,1.366693,0.878032,0.878032,Same as average of surface primary


##### 2.2. Average consumption per type of surface

In [47]:
subset_fuel_consumption_surface_type = subset_fuel_consumption.groupby(['surface_type']).sum()[['delta_fuel', 'km']].reset_index()

C:\Users\garji\AppData\Local\Temp\ipykernel_5924\1815888095.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subset_fuel_consumption_surface_type = subset_fuel_consumption.groupby(['surface_type']).sum()[['delta_fuel', 'km']].reset_index()


In [48]:
subset_fuel_consumption_surface_type['consumption'] = subset_fuel_consumption_surface_type['delta_fuel'] / subset_fuel_consumption_surface_type['km']

In [49]:
a = subset_fuel_consumption.copy()
a = a.merge(subset_fuel_consumption_surface_type[['surface_type', 'consumption']], how = 'inner', left_on = ['surface_type'], right_on = ['surface_type'])

In [50]:
a['comparison_overall'] = a['consumption_x'].apply(lambda x: 'Greater than average' if x > avg_consumption 
                                                                                    else (
                                                                                        'Same as average' if x == avg_consumption 
                                                                                                          else 'Lower than average'))

In [51]:
a['comparison_per_road_type'] = a[['surface_type','consumption_x', 'consumption_y']].apply(lambda x: 'Greater than average of surface {}'.format(x['surface_type'])
                                                                                           if x['consumption_x'] > x['consumption_y'] 
                                                                                                   else (
                                                                                                    'Same as average of surface {}'.format(x['surface_type'])
                                                                                                       if x['consumption_x'] == x['consumption_y'] 
                                                                                                       else 'Lower than average of surface {}'.format(x['surface_type']))
                                                                                           , axis = 1)

In [52]:
a

,id,vehicle_id,unix_timestamp,latitude,longitude,vehicle_speed,fuel_level,road_type,surface_type,elevation,next_unix_timestamp,previous_fuel_level,delta_seconds,delta_fuel,km,consumption_x,consumption_y,comparison_overall,comparison_per_road_type
0,16,f5uo7nhgj,1663261586,55.042694,11.987868,83.168006,48.4,tertiary,asphalt,44,1.663262e+09,48.8,117.0,0.4,2.702960,0.147986,0.246941,Lower than average,Lower than average of surface asphalt
1,42,478v3j0qq,1663261585,52.683624,13.343811,81.124029,16.4,motorway,asphalt,72,1.663262e+09,17.2,123.0,0.8,2.771738,0.288628,0.246941,Greater than average,Greater than average of surface asphalt
2,45,lta7bpqtk,1663261585,53.549400,10.688777,83.541390,25.6,motorway,asphalt,19,1.663262e+09,25.6,124.0,0.0,2.877537,0.000000,0.246941,Lower than average,Lower than average of surface asphalt
3,56,rg1f1f4wq,1663261588,51.448326,6.740520,41.345348,53.6,primary,asphalt,27,1.663262e+09,54.8,119.0,1.2,1.366693,0.878032,0.246941,Greater than average,Greater than average of surface asphalt
4,27,pqf1ffjoi,1663261585,51.933884,4.074568,81.000854,53.6,service,paved,8,1.663262e+09,53.6,123.0,0.0,2.767529,0.000000,0.000000,Lower than average,Same as average of surface paved


#### 3. No of trips made

To consider a trip:
- the car must have drive at a speed different than 0
- if a car in the overall trip stops for more than 10 minutes, then it'd be considered a different trip
- if more than 30 minutes passes between two consecutive timestamps given the previous conditions, then it'd be considered a new trip

In [53]:
df_trips = df[['id', 'vehicle_id', 'unix_timestamp', 'vehicle_speed']].copy()

In [54]:
drop_stopped_vehicles = df_trips.groupby(['vehicle_id']).max()['vehicle_speed'].reset_index()
drop_stopped_vehicles = drop_stopped_vehicles[drop_stopped_vehicles['vehicle_speed'] != 0]['vehicle_id'].unique()

In [55]:
df_trips = df_trips[df_trips['vehicle_id'].isin(drop_stopped_vehicles)]

In [56]:
df_trips['next_timestamp'] = df_trips.groupby(['vehicle_id'])['unix_timestamp'].shift(-1) - 1

In [57]:
df_trips['seconds'] = df_trips['next_timestamp'] - df_trips['unix_timestamp']

In [58]:
drop_stopped_vehicles_more_10_minutes = df_trips[df_trips['vehicle_speed'] == 0].groupby(['vehicle_id']).sum()['seconds'].reset_index()
drop_stopped_vehicles_more_10_minutes = drop_stopped_vehicles_more_10_minutes[drop_stopped_vehicles_more_10_minutes['seconds']/60 > 10]['vehicle_id'].unique()

In [59]:
df_trips = df_trips[~(df_trips['vehicle_id'].isin(drop_stopped_vehicles_more_10_minutes) &
                   (df_trips['vehicle_speed'] == 0)) &
                   (df_trips['vehicle_speed'] > 0)]

In [60]:
df_trips['is_new_trip'] = df_trips['seconds'].apply(lambda x: True if x > 1800 else False)

In [61]:
total_trips = df_trips.groupby(['vehicle_id']).count()['is_new_trip'].value_counts().reset_index()

In [62]:
total_trips = sum(total_trips['is_new_trip']) 

In [63]:
print('Total number of trips made: {}'.format(total_trips))

Total number of trips made: 39


#### 4. Trips with most economical fuel consumption behaviours

As each car considered in the analysis has made only one trip based on our conditions, then we can calculate the consumption per vehicle_id as previously done in other sections.

In [64]:
df_trips = df[['id', 'vehicle_id', 'unix_timestamp', 'vehicle_speed', 'fuel_level']].copy()

In [65]:
drop_stopped_vehicles = df_trips.groupby(['vehicle_id']).max()['vehicle_speed'].reset_index()
drop_stopped_vehicles = drop_stopped_vehicles[drop_stopped_vehicles['vehicle_speed'] != 0]['vehicle_id'].unique()

In [66]:
df_trips = df_trips[df_trips['vehicle_id'].isin(drop_stopped_vehicles)]

In [67]:
df_trips['next_timestamp'] = df_trips.groupby(['vehicle_id'])['unix_timestamp'].shift(-1) - 1

In [68]:
df_trips['seconds'] = df_trips['next_timestamp'] - df_trips['unix_timestamp']

In [69]:
drop_stopped_vehicles_more_10_minutes = df_trips[df_trips['vehicle_speed'] == 0].groupby(['vehicle_id']).sum()['seconds'].reset_index()
drop_stopped_vehicles_more_10_minutes = drop_stopped_vehicles_more_10_minutes[drop_stopped_vehicles_more_10_minutes['seconds']/60 > 10]['vehicle_id'].unique()

In [70]:
df_trips = df_trips[~(df_trips['vehicle_id'].isin(drop_stopped_vehicles_more_10_minutes) &
                   (df_trips['vehicle_speed'] == 0)) &
                   (df_trips['vehicle_speed'] > 0)]

In [71]:
df_trips['is_new_trip'] = df_trips['seconds'].apply(lambda x: True if x > 1800 else False)

In [72]:
changed_fuel_level = df_trips.groupby(['vehicle_id']).nunique()['fuel_level'].reset_index()
changed_fuel_level = changed_fuel_level[changed_fuel_level['fuel_level'] > 1]
df_trips = df_trips[df_trips['vehicle_id'].isin(changed_fuel_level['vehicle_id'].unique())]

In [73]:
df_trips['previous_fuel_level'] = df_trips.groupby(['vehicle_id']).shift(1)['fuel_level']

In [74]:
df_trips['delta_fuel_level'] = df_trips['previous_fuel_level'] - df_trips['fuel_level'] 

In [75]:
df_trips['km'] = df_trips['vehicle_speed'] * df_trips['seconds'] / 3600

# This dataset will be useful for the 5th question
kilometers_per_vehicle = df_trips.groupby(['vehicle_id']).sum()['km'].reset_index()

In [76]:
df_trips = df_trips[(df_trips['next_timestamp'] > df_trips['unix_timestamp']) &
                    (df_trips['fuel_level'] <= df_trips['previous_fuel_level'])
                   ]

In [77]:
df_trips = df_trips.groupby(['vehicle_id']).sum()[['delta_fuel_level', 'km']]

In [78]:
df_trips['consumption'] = df_trips['delta_fuel_level'] / df_trips['km']

In [79]:
df_trips.sort_values(by=['consumption']).head(10)

,delta_fuel_level,km,consumption
vehicle_id,,,
lta7bpqtk,0.0,2.854331,0.000000
pqf1ffjoi,0.0,2.745029,0.000000
f5uo7nhgj,0.4,2.679858,0.149262
478v3j0qq,0.8,2.749203,0.290993
rg1f1f4wq,1.2,1.355209,0.885473


#### 5. Usage profile of the vehicle

Usage profile determined by:
- Total kilometers done
- General consumption of the vehicle
- Richness of information
- Usual road and type of surface
- Average elevation
- Informational quality (number of missing fields)

Total kilometers done:

In [80]:
vehicles = pd.DataFrame(df['vehicle_id'].unique())

In [81]:
vehicles.columns = ['vehicle_id']

In [82]:
vehicles = vehicles.merge(kilometers_per_vehicle, how = 'left', left_on = ['vehicle_id'], right_on = ['vehicle_id'])

In [83]:
vehicles['kilometers_qualitative'] = vehicles['km'].apply(lambda x: 'Above median km' if x > np.median(kilometers_per_vehicle['km'])
                                                                                      else ('Median' if x == np.median(kilometers_per_vehicle['km'])
                                                                                                    else ('No data' if pd.isnull(x) == True else 'Below median km')
                                                                                           )   
                                                         )

General consumption of the vehicle:

In [84]:
vehicles = vehicles.merge(subset_fuel_consumption[['vehicle_id', 'consumption']], how = 'left', left_on = ['vehicle_id'], right_on = ['vehicle_id'])
vehicles['consumption_qualitative'] = vehicles['consumption'].apply(lambda x: 'Above median consumption' if x > np.median(subset_fuel_consumption['consumption'])
                                                                              else ('Median' if x == np.median(subset_fuel_consumption['consumption'])
                                                                                  else ('No data' if pd.isnull(x) == True else 'Below median consumption')
                                                                                   )   
                                                                   )

Usual roads and surface types:

In [85]:
a = df.groupby(['vehicle_id', 'road_type']).count()['id'].reset_index().set_index(['vehicle_id', 'id'])
b = a.reset_index().groupby(['vehicle_id']).max()['id'].reset_index().set_index(['vehicle_id', 'id'])

a = a[a.index.isin(b.index)].reset_index()

In [86]:
a = a.groupby(['vehicle_id']).agg(set)['road_type'].reset_index()

In [87]:
vehicles = vehicles.merge(a, how = 'left', left_on = ['vehicle_id'], right_on = ['vehicle_id'])

In [88]:
a = df.groupby(['vehicle_id', 'surface_type']).count()['id'].reset_index().set_index(['vehicle_id', 'id'])
b = a.reset_index().groupby(['vehicle_id']).max()['id'].reset_index().set_index(['vehicle_id', 'id'])

a = a[a.index.isin(b.index)].reset_index()

In [89]:
a = a.groupby(['vehicle_id']).agg(set)['surface_type'].reset_index()

In [90]:
vehicles = vehicles.merge(a, how = 'left', left_on = ['vehicle_id'], right_on = ['vehicle_id'])

Average elevation (I checked for this section that the mean, minimum and maximum average of the difference between the mean and median of the elevation is below the 10%, so that's why I chese to take the average elevation):

In [91]:
a = df[['vehicle_id', 'elevation']].groupby(['vehicle_id']).mean().reset_index()
b = df[['vehicle_id', 'elevation']].groupby(['vehicle_id']).median().reset_index()

In [92]:
vehicles = vehicles.merge(a, how = 'inner', left_on = ['vehicle_id'], right_on = ['vehicle_id'])
vehicles = vehicles.merge(b, how = 'inner', left_on = ['vehicle_id'], right_on = ['vehicle_id'])

In [93]:
vehicles.rename(columns={'elevation_x':'avg_elevation', 'elevation_y':'median_elevation'}, inplace = True)

In [94]:
vehicles['dev'] = (vehicles['avg_elevation'] - vehicles['median_elevation']) * 100 / vehicles['avg_elevation']

In [95]:
print(np.mean(vehicles['dev']), np.min(vehicles['dev']), np.max(vehicles['dev']))

-0.4441486151054181 -7.368421052631574 5.813953488372097


In [96]:
vehicles.drop(columns=['dev', 'median_elevation'], inplace = True)

In [97]:
vehicles['elevation_qualitative'] = vehicles['avg_elevation'].apply(lambda x: 'Above avg elevation' if x > np.mean(vehicles['avg_elevation'])
                                                                                                    else ('Mean elevation' if x == np.mean(vehicles['avg_elevation'])
                                                                                                                           else 'Below avg elevation'
                                                                                                         )
                                                                   )

Informational quality (number of missing fields):

In [98]:
informational_quality = pd.isnull(vehicles[['vehicle_id', 'km', 'consumption', 'road_type', 'surface_type', 'avg_elevation']].set_index(['vehicle_id'])).sum(axis = 1).reset_index()

In [99]:
informational_quality.columns = ['vehicle_id', 'missing_fields']

In [100]:
informational_quality['informational_quality'] = informational_quality['missing_fields'].apply(lambda x: 'No missing fields' if x == 0 
                                                                                               else ('Below median missing fields' if x < np.median(informational_quality['missing_fields'])
                                                                                                    else ('Above median missing fields' if x > np.median(informational_quality['missing_fields'])
                                                                                                          else 'Median missing fields')))

Final dataset:

In [101]:
vehicles = vehicles.merge(informational_quality, how = 'left', left_on = ['vehicle_id'], right_on = ['vehicle_id'])
vehicles

,vehicle_id,km,kilometers_qualitative,consumption,consumption_qualitative,road_type,surface_type,avg_elevation,elevation_qualitative,missing_fields,informational_quality
0,x1sdt0ier,NaN,No data,NaN,No data,{service},{asphalt},5.000000,Below avg elevation,2,Median missing fields
1,3o1e4e0qf,NaN,No data,NaN,No data,{service},{asphalt},5.750000,Below avg elevation,2,Median missing fields
2,3tq3s6p4g,NaN,No data,NaN,No data,{service},{asphalt},6.000000,Below avg elevation,2,Median missing fields
3,f5uo7nhgj,5.478428,Above median km,0.147986,Median,{motorway},{asphalt},54.000000,Above avg elevation,0,No missing fields
4,a0pyo73uo,NaN,No data,NaN,No data,{service},{asphalt},36.000000,Below avg elevation,2,Median missing fields
5,gakx7jksd,NaN,No data,NaN,No data,{service},{asphalt},21.333333,Below avg elevation,2,Median missing fields
6,pqf1ffjoi,5.132575,Below median km,0.000000,Below median consumption,{service},{asphalt},6.000000,Below avg elevation,0,No missing fields
7,vs417ktgc,NaN,No data,NaN,No data,{unclassified},{asphalt},16.000000,Below avg elevation,2,Median missing fields
8,92ebhgzjs,NaN,No data,NaN,No data,{service},{asphalt},2.000000,Below avg elevation,2,Median missing fields
9,gupxdhzch,NaN,No data,NaN,No data,{motorway},{asphalt},75.750000,Above avg elevation,2,Median missing fields
